<a href="https://colab.research.google.com/github/JK-the-Ko/Thermo-Fluid-Dynamics-Experiment/blob/main/2022-2/%EC%97%B4%EC%9C%A0%EC%B2%B4%EA%B3%B5%ED%95%99%EC%8B%A4%ED%97%98_Week_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CFD Analysis with Deep Neural Network (DNN)

# Dataset

## Download Dataset

In [ ]:
!wget https://zenodo.org/record/3666056/files/DeepCFD.zip?download=1

## Unzip Dataset

In [ ]:
!mv "/content/DeepCFD.zip?download=1" "/content/DeepCFD.zip"

In [ ]:
!unzip "/content/DeepCFD.zip" -d "/content"

## Analyze Dataset

In [ ]:
import pickle

In [ ]:
x = pickle.load(open("/content/dataX.pkl", "rb"))
y = pickle.load(open("/content/dataY.pkl", "rb"))

In [ ]:
type(x)

In [ ]:
x.shape

In [ ]:
y.shape

### Preprocess Dataset

In [ ]:
import numpy as np

In [ ]:
x = np.transpose(x, (0, 3, 2, 1))
y = np.transpose(y, (0, 3, 2, 1))

In [ ]:
x.shape

In [ ]:
y.shape

## Visualize Dataset

In [ ]:
import matplotlib.pyplot as plt

### Input Data

In [ ]:
plt.figure(figsize=(30, 60))

plt.subplot(131)
plt.imshow(x[0,:,:,0])

plt.subplot(132)
plt.imshow(x[0,:,:,1])

plt.subplot(133)
plt.imshow(x[0,:,:,2])

plt.show()

### Target Data

In [ ]:
plt.figure(figsize=(30, 60))

plt.subplot(131)
plt.imshow(y[0,:,:,0])

plt.subplot(132)
plt.imshow(y[0,:,:,1])

plt.subplot(133)
plt.imshow(y[0,:,:,2])

plt.show()

### Input Data

In [ ]:
plt.figure(figsize=(30, 60))

plt.subplot(131)
plt.imshow(x[-1,:,:,0])

plt.subplot(132)
plt.imshow(x[-1,:,:,1])

plt.subplot(133)
plt.imshow(x[-1,:,:,2])

plt.show()

### Target Data

In [ ]:
plt.figure(figsize=(30, 60))

plt.subplot(131)
plt.imshow(y[-1,:,:,0])

plt.subplot(132)
plt.imshow(y[-1,:,:,1])

plt.subplot(133)
plt.imshow(y[-1,:,:,2])

plt.show()

### Split Dataset

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.1, random_state=42)

In [ ]:
xTrain, xValid, yTrain, yValid = train_test_split(xTrain, yTrain, test_size=0.1, random_state=42)

In [ ]:
xTrain.shape

In [ ]:
xTest.shape

In [ ]:
yTrain.shape

In [ ]:
yTest.shape

## Build Convolutional Neural Network (CNN)

### Import Keras

In [ ]:
import keras
from keras.models import Model, load_model
from keras.layers import Input, concatenate, Resizing
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D

### Fix Seed

In [ ]:
import tensorflow as tf
from keras import backend as K
import random

def fixSeed(numSeed = 42) :
  np.random.seed(numSeed)
  random.seed(numSeed)
  tf.random.set_seed(numSeed)

  sessionConf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=sessionConf)
  K.set_session(sess)

### Build U-Net Network

In [ ]:
inputs = Input((x.shape[1], x.shape[2], x.shape[3]))

c1 = Conv2D(16, (3, 3), activation="elu", kernel_initializer="he_normal", padding="same")(inputs)
c1 = Dropout(0.1)(c1)
c1 = Conv2D(16, (3, 3), activation="elu", kernel_initializer="he_normal", padding="same")(c1)
p1 = MaxPooling2D((2, 2))(c1)

c2 = Conv2D(32, (3, 3), activation="elu", kernel_initializer="he_normal", padding="same")(p1)
c2 = Dropout(0.1)(c2)
c2 = Conv2D(32, (3, 3), activation="elu", kernel_initializer="he_normal", padding="same")(c2)
p2 = MaxPooling2D((2, 2))(c2)

c3 = Conv2D(64, (3, 3), activation="elu", kernel_initializer="he_normal", padding="same")(p2)
c3 = Dropout(0.2)(c3)
c3 = Conv2D(64, (3, 3), activation="elu", kernel_initializer="he_normal", padding="same")(c3)
p3 = MaxPooling2D((2, 2))(c3)

c4 = Conv2D(128, (3, 3), activation="elu", kernel_initializer="he_normal", padding="same")(p3)
c4 = Dropout(0.2)(c4)
c4 = Conv2D(128, (3, 3), activation="elu", kernel_initializer="he_normal", padding="same")(c4)
p4 = MaxPooling2D(pool_size=(2, 2))(c4)

c5 = Conv2D(256, (3, 3), activation="elu", kernel_initializer="he_normal", padding="same")(p4)
c5 = Dropout(0.3)(c5)
c5 = Conv2D(256, (3, 3), activation="elu", kernel_initializer="he_normal", padding="same")(c5)

u6 = Resizing(c4.shape[1], c4.shape[2], "nearest")(c5)
u6 = Conv2D(128, (3, 3), activation="elu", kernel_initializer="he_normal", padding="same")(u6)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation="elu", kernel_initializer="he_normal", padding="same")(u6)
c6 = Dropout(0.2)(c6)
c6 = Conv2D(128, (3, 3), activation="elu", kernel_initializer="he_normal", padding="same")(c6)

u7 = Resizing(c3.shape[1], c3.shape[2], "nearest")(c6)
u7 = Conv2D(64, (3, 3), activation="elu", kernel_initializer="he_normal", padding="same")(u7)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation="elu", kernel_initializer="he_normal", padding="same")(u7)
c7 = Dropout(0.2)(c7)
c7 = Conv2D(64, (3, 3), activation="elu", kernel_initializer="he_normal", padding="same")(c7)

u8 = Resizing(c2.shape[1], c2.shape[2], "nearest")(c7)
u8 = Conv2D(32, (3, 3), activation="elu", kernel_initializer="he_normal", padding="same")(u8)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation="elu", kernel_initializer="he_normal", padding="same")(u8)
c8 = Dropout(0.1)(c8)
c8 = Conv2D(32, (3, 3), activation="elu", kernel_initializer="he_normal", padding="same")(c8)

u9 = Resizing(c1.shape[1], c1.shape[2], "nearest")(c8)
u8 = Conv2D(16, (3, 3), activation="elu", kernel_initializer="he_normal", padding="same")(u9)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation="elu", kernel_initializer="he_normal", padding="same")(u9)
c9 = Dropout(0.1)(c9)
c9 = Conv2D(16, (3, 3), activation="elu", kernel_initializer="he_normal", padding="same")(c9)

outputs = Conv2D(y.shape[-1], (1, 1))(c9)

### Create Model Instance

In [ ]:
K.clear_session()
fixSeed()

model = Model(inputs=[inputs], outputs=[outputs])

### Summarize Model

In [ ]:
model.summary()

### Visualize Model

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file="model.png")

### Compile Model

In [ ]:
lr, batchSize, epoch = 1e-3, 8, 50

In [ ]:
model.compile(loss="mean_absolute_error", optimizer=keras.optimizers.Adam(learning_rate=lr))

### Train Model

In [ ]:
history = model.fit(xTrain, yTrain, batch_size=batchSize, epochs=epoch, validation_data=(xValid, yValid))

### Visualize Training Procedure

In [ ]:
plt.subplots(figsize = (20,5))
plt.plot(np.arange(epoch), history.history["loss"], label="Training MAE Loss")
plt.plot(np.arange(epoch), history.history["val_loss"], label="Validation MAE Loss")
plt.xlabel("# Epoch")
plt.ylabel("MAE Loss")
plt.title("Loss")
plt.legend(loc="best")
plt.show()

### Inference Result

In [ ]:
yPred = model.predict(xTest)

In [ ]:
yPred.shape

### Reshape Array

In [ ]:
yTest = np.transpose(yTest, (0, 3, 2, 1))
yPred = np.transpose(yPred, (0, 3, 2, 1))

In [ ]:
yTest.shape

In [ ]:
yPred.shape

### Visualize Result

In [ ]:
def visualize(sample_y, out_y, error, s) :
    minu = np.min(sample_y[s, 0, :, :])
    maxu = np.max(sample_y[s, 0, :, :])
    
    minv = np.min(sample_y[s, 1, :, :])
    maxv = np.max(sample_y[s, 1, :, :])
    
    minp = np.min(sample_y[s, 2, :, :])
    maxp = np.max(sample_y[s, 2, :, :])
    
    mineu = np.min(error[s, 0, :, :])
    maxeu = np.max(error[s, 0, :, :])
    
    minev = np.min(error[s, 1, :, :])
    maxev = np.max(error[s, 1, :, :])
    
    minep = np.min(error[s, 2, :, :])
    maxep = np.max(error[s, 2, :, :])
           
    plt.figure()
    fig = plt.gcf()
    fig.set_size_inches(15, 10)
    plt.subplot(3, 3, 1)
    plt.title("CFD", fontsize=18)
    plt.imshow(np.transpose(sample_y[s, 0, :, :]), cmap="jet", vmin = minu, vmax = maxu, origin="lower", extent=[0,260,0,120])
    plt.colorbar(orientation="horizontal")
    plt.ylabel("Ux", fontsize=18)
    plt.subplot(3, 3, 2)
    plt.title("CNN", fontsize=18)
    plt.imshow(np.transpose(out_y[s, 0, :, :]), cmap="jet", vmin = minu, vmax = maxu, origin="lower", extent=[0,260,0,120])
    plt.colorbar(orientation="horizontal")
    plt.subplot(3, 3, 3)
    plt.title("Error", fontsize=18)
    plt.imshow(np.transpose(error[s, 0, :, :]), cmap="jet", vmin = mineu, vmax = maxeu, origin="lower", extent=[0,260,0,120])
    plt.colorbar(orientation="horizontal")

    plt.subplot(3, 3, 4)
    plt.imshow(np.transpose(sample_y[s, 1, :, :]), cmap="jet", vmin = minv, vmax = maxv, origin="lower", extent=[0,260,0,120])
    plt.colorbar(orientation="horizontal")
    plt.ylabel("Uy", fontsize=18)
    plt.subplot(3, 3, 5)
    plt.imshow(np.transpose(out_y[s, 1, :, :]), cmap="jet", vmin = minv, vmax = maxv, origin="lower", extent=[0,260,0,120])
    plt.colorbar(orientation="horizontal")
    plt.subplot(3, 3, 6)
    plt.imshow(np.transpose(error[s, 1, :, :]), cmap="jet", vmin = minev, vmax = maxev, origin="lower", extent=[0,260,0,120])
    plt.colorbar(orientation="horizontal")

    plt.subplot(3, 3, 7)
    plt.imshow(np.transpose(sample_y[s, 2, :, :]), cmap="jet", vmin = minp, vmax = maxp, origin="lower", extent=[0,260,0,120])
    plt.colorbar(orientation="horizontal")
    plt.ylabel("p", fontsize=18)
    plt.subplot(3, 3, 8)
    plt.imshow(np.transpose(out_y[s, 2, :, :]), cmap="jet", vmin = minp, vmax = maxp, origin="lower", extent=[0,260,0,120])
    plt.colorbar(orientation="horizontal")
    plt.subplot(3, 3, 9)
    plt.imshow(np.transpose(error[s, 2, :, :]), cmap="jet", vmin = minep, vmax = maxep, origin="lower", extent=[0,260,0,120])
    plt.colorbar(orientation="horizontal")
    plt.tight_layout()
    plt.show()

In [ ]:
error = np.abs(yPred-yTest)

In [ ]:
visualize(yTest, yPred, error, 0)

In [ ]:
visualize(yTest, yPred, error, 30)

In [ ]:
visualize(yTest, yPred, error, -1)